In [2]:
import io
from IPython.nbformat import current
def execute_notebook(nbfile):
    with io.open(nbfile) as f:
        nb = current.read(f, 'json')
    ip = get_ipython()
    for cell in nb.worksheets[0].cells:
        if cell.cell_type != 'code':
            continue
        ip.run_cell(cell.input)
execute_notebook("/cellar/users/ramarty/Projects/kir/KIR_development/bin/imports.ipynb")

/cellar/users/ramarty/anaconda/lib/python2.7/site-packages/IPython/nbformat/current.py:19: UserWarning: IPython.nbformat.current is deprecated.

- use IPython.nbformat for read/write/validate public API
- use IPython.nbformat.vX directly to composing notebooks of a particular version

  """)
:0: FutureWarning: IPython widgets are experimental and may change in the future.


Populating the interactive namespace from numpy and matplotlib


###Acquire samples to run

In [ ]:
out_dirs = ['/nrnb/users/ramarty/kir/TCGA/exomes/{0}'.format(x) for x in normal_samples]

###Create cluster script

In [ ]:
def create_cluster_script_type(samples, out_dirs):

    new_script_file = '/cellar/users/ramarty/Projects/kir/scripts/cluster/TCGA.component_collection.sh'

    with open(new_script_file, 'w') as out_file:
        out_file.write("#! /bin/csh\n")
        out_file.write("#$ -V\n")
        out_file.write("#$ -S /bin/csh\n")
        out_file.write("#$ -o /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -e /cellar/users/ramarty/Data/kir/sge-system_files\n")
        out_file.write("#$ -cwd\n")
        out_file.write("#$ -t 1-{0}\n".format(len(samples)))
        #out_file.write("#$ -t 1-10\n".format(len(samples)))
        out_file.write("#$ -l h_vmem=10G\n")
        out_file.write("#$ -tc 200\n")
        out_file.write("#$ -l long")
        out_file.write("\n")

        out_file.write("set samples=({0})\n".format(" ".join(samples)))
        out_file.write("set outs=({0})\n".format(" ".join(out_dirs)))
        out_file.write("\n")

        out_file.write("set sample=$samples[$SGE_TASK_ID]\n")
        out_file.write("set out=$outs[$SGE_TASK_ID]\n")
        out_file.write("\n")

        out_file.write("date\n")
        out_file.write("hostname\n")
        out_file.write("mkdir $out\n")
        out_file.write("mkdir $out/components\n")
        out_file.write("bash /nrnb/users/ramarty/kir/scripts/GDC.exome.sh $sample $out/full_exome.bam\n")
        out_file.write("python /cellar/users/ramarty/Projects/kir/scripts/python/convert_to_fastq.py $out/full_exome.bam $out/full_exome.fastq\n")
        out_file.write("python /cellar/users/ramarty/Projects/kir/scripts/python/component_collection.py $out/full_exome.fastq $out/components\n")
        out_file.write("rm $out/full_exome.fastq\n")
        out_file.write("rm $out/full_exome.bam\n")
        out_file.write("date\n")